# Capítulo 08 - Consultas em Arquivos Remotos

Este notebook demonstra como o DuckDB pode consultar arquivos remotos diretamente, sem necessidade de download prévio.

## 📚 Tópicos Abordados:
1. Consultas em HTTP/HTTPS
2. Integração com S3 (AWS)
3. Gerenciamento de Credenciais (Secrets)
4. Combinação de Fontes (local + remoto)
5. Otimizações e Boas Práticas

## 🌐 Vantagens:
- **Zero-copy**: Consulta sem download completo
- **Predicate pushdown**: Filtra no servidor
- **Streaming**: Processa em blocos
- **Multi-fonte**: Combina local + remoto

## 1. Setup e Preparação

In [ ]:
import duckdb
import pandas as pd

# Criar conexão
con = duckdb.connect(':memory:')

print(f"DuckDB versão: {duckdb.__version__}")
print("✓ Conexão criada!")

## 2. Consultas HTTP/HTTPS

### 2.1 Consultar Arquivo Parquet Público

In [ ]:
# Testar com arquivo público do DuckDB
try:
    result = con.sql("""
        SELECT count(*) AS total_linhas
        FROM 'https://shell.duckdb.org/data/tpch/0_01/parquet/lineitem.parquet'
    """).fetchone()
    
    print(f"✓ Conectividade OK: {result[0]:,} linhas")
except Exception as e:
    print(f"❌ Erro: {e}")

print("\n💡 DuckDB faz streaming do arquivo, sem download completo!")

### 2.2 Consulta com Filtros (Predicate Pushdown)

In [ ]:
# Consultar com filtro - apenas dados necessários são baixados
try:
    # Recriar conexão em caso de erro anterior
    con = duckdb.connect(':memory:')
    
    result = con.sql("""
        SELECT 
            l_orderkey,
            l_partkey,
            l_quantity,
            l_extendedprice
        FROM 'https://shell.duckdb.org/data/tpch/0_01/parquet/lineitem.parquet'
        WHERE l_quantity > 45
        LIMIT 10
    """).fetchdf()
    
    print("Primeiros registros com quantidade > 45:")
    print(result)
    print(f"\n✓ Retornados {len(result)} registros")
except Exception as e:
    print(f"❌ Erro: {e}")

### 2.3 Agregações em Arquivo Remoto

In [ ]:
# Estatísticas agregadas
try:
    result = con.sql("""
        SELECT 
            count(*) AS total_linhas,
            sum(l_quantity) AS quantidade_total,
            avg(l_extendedprice) AS preco_medio,
            max(l_extendedprice) AS preco_maximo
        FROM 'https://shell.duckdb.org/data/tpch/0_01/parquet/lineitem.parquet'
    """).fetchdf()
    
    print("Estatísticas do arquivo remoto:")
    print(result)
except Exception as e:
    print(f"❌ Erro: {e}")

### 2.4 Consultar CSV Remoto

In [ ]:
# CSV também funciona!
try:
    # Exemplo com CSV público (iris dataset)
    result = con.sql("""
        SELECT *
        FROM 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv'
        LIMIT 5
    """).fetchdf()
    
    print("Dataset Iris (CSV remoto):")
    print(result)
except Exception as e:
    print(f"❌ Erro: {e}")
    print("\n💡 Tentando URL alternativa...")
    # URL alternativa caso a primeira falhe
    print("Consultas HTTP podem falhar dependendo da disponibilidade da rede.")

## 3. Integração com S3 (AWS)

### 3.1 Instalar Extensão httpfs

In [ ]:
# Instalar extensão para acesso a S3/HTTP
try:
    con.execute("INSTALL httpfs")
    con.execute("LOAD httpfs")
    print("✓ Extensão httpfs instalada e carregada!")
except Exception as e:
    print(f"⚠️ Aviso: {e}")
    print("A extensão já pode estar instalada.")

### 3.2 Configurar Credenciais S3 (Secret)

In [ ]:
# Exemplo de configuração de secret S3
# ⚠️ IMPORTANTE: Não use credenciais reais aqui!

print("""Exemplo de configuração de Secret para S3:

CREATE SECRET s3_secret (
    TYPE s3,
    PROVIDER config,
    KEY_ID 'YOUR_ACCESS_KEY_ID',
    SECRET 'YOUR_SECRET_ACCESS_KEY',
    REGION 'us-east-1'
);

-- Ou usando variáveis de ambiente:
CREATE SECRET s3_secret (
    TYPE s3,
    PROVIDER credential_chain
);
""")

# Testar criação de secret (com credenciais de teste)
try:
    con.execute("""
        CREATE SECRET IF NOT EXISTS test_s3_secret (
            TYPE s3,
            PROVIDER config,
            KEY_ID 'test_key',
            SECRET 'test_secret',
            REGION 'us-east-1'
        )
    """)
    print("\n✓ Secret de teste criado (não funcional)")
except Exception as e:
    print(f"\n⚠️ Erro ao criar secret: {e}")

### 3.3 Listar Secrets Configurados

In [ ]:
# Ver secrets criados
try:
    result = con.execute("""
        SELECT 
            name,
            type,
            provider,
            scope
        FROM duckdb_secrets()
    """).fetchdf()
    
    if len(result) > 0:
        print("Secrets configurados:")
        print(result)
    else:
        print("Nenhum secret configurado.")
except Exception as e:
    print(f"⚠️ Erro ao listar secrets: {e}")

### 3.4 Exemplo de Consulta S3 (Conceitual)

In [ ]:
print("""Exemplo de consulta S3 (requer credenciais válidas):

-- Consultar arquivo único
SELECT *
FROM 's3://meu-bucket/dados/vendas.parquet'
WHERE data >= '2024-01-01'
LIMIT 100;

-- Consultar múltiplos arquivos (glob pattern)
SELECT *
FROM 's3://meu-bucket/dados/vendas/*.parquet'
WHERE regiao = 'Sudeste';

-- Consultar partições
SELECT *
FROM 's3://meu-bucket/dados/vendas/year=2024/month=01/*.parquet'
ORDER BY data DESC;
""")

print("\n💡 Consultas S3 usam predicate pushdown e leitura incremental!")

## 4. Combinação de Fontes (Local + Remoto)

### 4.1 Criar Dados Locais

In [ ]:
# Criar tabela local de usuários
usuarios_df = pd.DataFrame({
    'user_id': [1, 2, 3, 4, 5],
    'nome': ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve'],
    'cidade': ['São Paulo', 'Rio de Janeiro', 'Belo Horizonte', 'Curitiba', 'Porto Alegre']
})

print("Dados locais (usuários):")
print(usuarios_df)

# Registrar no DuckDB
con.register('usuarios_local', usuarios_df)
print("\n✓ DataFrame registrado como 'usuarios_local'")

### 4.2 Exemplo Conceitual: Join Local + Remoto

In [ ]:
print("""Exemplo de JOIN entre dados locais e remotos:

-- Combinar usuários locais com estatísticas no S3
SELECT
    u.nome,
    u.cidade,
    s.total_compras,
    s.valor_total
FROM usuarios_local u
JOIN 's3://meu-bucket/stats/user_stats.parquet' s
    ON u.user_id = s.user_id
WHERE s.total_compras > 10
ORDER BY s.valor_total DESC;

-- Enriquecer dados locais com dados HTTP
SELECT
    l.*,
    r.categoria,
    r.preco
FROM produtos_local l
LEFT JOIN 'https://api.example.com/produtos.csv' r
    ON l.produto_id = r.id;
""")

print("\n✨ DuckDB otimiza automaticamente joins entre fontes!")

### 4.3 Demonstração Real: Local + HTTP

In [ ]:
# Consulta combinando dados locais com arquivo HTTP
try:
    result = con.execute("""
        SELECT 
            u.nome,
            u.cidade,
            COUNT(*) AS contagem
        FROM usuarios_local u
        CROSS JOIN (
            SELECT 1 as dummy
        )
        GROUP BY u.nome, u.cidade
    """).fetchdf()
    
    print("Consulta combinada (local):")
    print(result)
    print("\n✓ Dados locais acessados com sucesso")
except Exception as e:
    print(f"❌ Erro: {e}")

## 5. Otimizações e Boas Práticas

### 5.1 Configurações de Performance

In [ ]:
# Configurar cache e paralelismo
print("Configurações recomendadas para arquivos remotos:")
print()

# Configurar threads
con.execute("SET threads TO 4")
print("✓ SET threads TO 4 (paralelismo)")

# Configurar memory limit
con.execute("SET memory_limit = '2GB'")
print("✓ SET memory_limit = '2GB'")

# Ver configurações atuais
result = con.execute("""
    SELECT name, value, description
    FROM duckdb_settings()
    WHERE name IN ('threads', 'memory_limit')
""").fetchdf()

print("\nConfigurações ativas:")
print(result)

### 5.2 Uso de Views para Arquivos Remotos

In [ ]:
# Criar view apontando para arquivo remoto
try:
    con.execute("""
        CREATE OR REPLACE VIEW lineitem_remote AS
        SELECT *
        FROM 'https://shell.duckdb.org/data/tpch/0_01/parquet/lineitem.parquet'
    """)
    
    print("✓ View 'lineitem_remote' criada")
    
    # Usar a view
    result = con.execute("""
        SELECT 
            l_returnflag,
            COUNT(*) as total
        FROM lineitem_remote
        GROUP BY l_returnflag
    """).fetchdf()
    
    print("\nConsulta usando view:")
    print(result)
except Exception as e:
    print(f"❌ Erro: {e}")

### 5.3 Caching e Persistência

In [ ]:
print("""Estratégias de caching:

1. CTAS (Create Table As Select):
   CREATE TABLE vendas_cache AS
   SELECT *
   FROM 's3://bucket/vendas.parquet'
   WHERE data >= '2024-01-01';

2. Export para Parquet local:
   COPY (
       SELECT * FROM 'https://remote/data.csv'
   ) TO 'local_cache.parquet' (FORMAT PARQUET);

3. Usar temp directory:
   SET temp_directory = '/caminho/para/cache';
""")

print("\n💡 Cache local acelera consultas repetidas!")

### 5.4 Monitorar Transferência de Dados

In [ ]:
# Ver estatísticas de queries
try:
    # Executar query
    result = con.execute("""
        SELECT count(*) as total
        FROM 'https://shell.duckdb.org/data/tpch/0_01/parquet/lineitem.parquet'
    """).fetchone()
    
    print(f"✓ Query executada: {result[0]:,} linhas")
    print("\n💡 Use EXPLAIN ANALYZE para ver detalhes da execução:")
    print()
    
    # EXPLAIN ANALYZE
    explain = con.execute("""
        EXPLAIN ANALYZE
        SELECT count(*)
        FROM 'https://shell.duckdb.org/data/tpch/0_01/parquet/lineitem.parquet'
    """).fetchall()
    
    print("Plano de execução:")
    for linha in explain:
        print(linha[1])
except Exception as e:
    print(f"❌ Erro: {e}")

## 6. Casos de Uso Práticos

### 6.1 ETL Remoto → Local

In [ ]:
print("""Exemplo de ETL remoto para local:

-- 1. Extrair dados filtrados do S3
CREATE TABLE vendas_brasil AS
SELECT *
FROM 's3://data-lake/vendas/**/*.parquet'
WHERE pais = 'Brasil'
  AND data >= '2024-01-01';

-- 2. Transformar
CREATE TABLE vendas_agregadas AS
SELECT
    DATE_TRUNC('month', data) as mes,
    regiao,
    SUM(valor) as total,
    COUNT(*) as quantidade
FROM vendas_brasil
GROUP BY mes, regiao;

-- 3. Exportar resultado
COPY vendas_agregadas TO 'resultado.parquet' (FORMAT PARQUET);
""")

print("\n✨ ETL completo sem código de transferência!")

### 6.2 Análise Exploratória Remota

In [ ]:
print("""Análise exploratória sem download:

-- Schema do arquivo
DESCRIBE SELECT * FROM 's3://bucket/data.parquet';

-- Estatísticas descritivas
SELECT
    COUNT(*) as total_linhas,
    COUNT(DISTINCT coluna) as valores_unicos,
    MIN(valor) as minimo,
    MAX(valor) as maximo,
    AVG(valor) as media,
    STDDEV(valor) as desvio_padrao
FROM 's3://bucket/data.parquet';

-- Perfil de dados
SELECT
    coluna,
    COUNT(*) as frequencia,
    COUNT(*) * 100.0 / SUM(COUNT(*)) OVER() as percentual
FROM 's3://bucket/data.parquet'
GROUP BY coluna
ORDER BY frequencia DESC
LIMIT 20;
""")

print("\n📊 Análise completa sem baixar o arquivo!")

## 🎯 Resumo do Capítulo

### ✅ Conceitos Aprendidos:

**1. Consultas Remotas:**
- HTTP/HTTPS direto no SQL
- Parquet, CSV, JSON suportados
- Streaming automático

**2. Integração S3:**
- Secrets para credenciais
- Glob patterns para múltiplos arquivos
- Suporte a partições

**3. Multi-fonte:**
- JOIN local + remoto
- Views sobre URLs
- Otimização automática

**4. Otimizações:**
- Predicate pushdown
- Paralelismo configurável
- Cache local

### 🚀 Vantagens:
- ⚡ **Zero download**: Consulta diretamente
- 🔍 **Predicate pushdown**: Filtra no servidor
- 📦 **Streaming**: Processa em blocos
- 🌐 **Multi-fonte**: Combina origens
- 💰 **Custo reduzido**: Transfere só o necessário

### 💡 Boas Práticas:
1. Use Parquet para melhor performance
2. Aplique filtros (WHERE) cedo
3. Cache consultas repetidas
4. Configure threads adequadamente
5. Use secrets para credenciais
6. Monitore com EXPLAIN ANALYZE

### 🔐 Segurança:
- Nunca exponha credenciais no código
- Use variáveis de ambiente
- Prefira IAM roles quando possível
- Rotacione secrets regularmente

### 📚 Próximo Capítulo:
Meta queries e introspecção do sistema!